# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [3]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [4]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [5]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [6]:
from sklearn.metrics import recall_score,f1_score, precision_score
def evaluation_report(gold, pred): 
    count = 0
    for i in gold:
        if i in pred:
            count += 1
    recall = count/len(gold)
    
    count = 0
    for i in pred:
        if i in gold:
            count += 1
    precision = count/len(pred)
    
    f1_score = 2*((precision * recall)/(precision + recall))
    print(f"recall: {100 *recall}%")    
    print(f"precision: {100 *precision:}%")    
    print(f"f1 score: {int(100 * f1_score)}%")

To test your code, you can run the following cell:

In [7]:
evaluation_report(set(range(3)), set(range(5)))

recall: 100.0%
precision: 60.0%
f1 score: 74%


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.
    
    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for _,sentence_id, _, beg, end, *_ in df.itertuples():
        yield sentence_id, beg, end


To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [9]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [10]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data
def valid(lable):
    return True

def predict(df):
    prev_id = -1
    for _,sentence_id, sentence, beg, end, *_ in df.itertuples():
        if prev_id != sentence_id:
            prev_id = sentence_id
            doc = nlp(sentence)
            for ent in doc.ents:
                if valid(ent.label_):
                    yield sentence_id, ent.start, ent.end


In [11]:
predicted = set(predict(df_dev))
evaluation_report(spans_dev_gold, predicted)
    

recall: 70.00169004563124%
precision: 51.530231400845985%
f1 score: 59%


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [ ]:
def evaluation(gold, pred): 
    falseNeg = [i for i in gold if i not in pred]
    falsePos = [i for i in pred if i not in gold]
    fn_indexer = [i[0] for i in falseNeg]  
    fp_indexer = [i[0] for i in falsePos] 
    return df_dev[df_dev['sentence_id'].isin(fn_indexer)].drop_duplicates(['sentence_id']), df_dev[df_dev['sentence_id'].isin(fp_indexer)].drop_duplicates(['sentence_id']) 

fn_df, fp_df = evaluation(spans_dev_gold, predicted)
display(fn_df.head(100), fp_df.head(100))

*TODO: Write a short text that discusses the errors that you observed*

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [14]:
# TODO: Write code here to improve the span prediction from Problem 2
def valid(lable):
    return lable not in ["CARDINAL", "PERCENT", "MONEY","QUANTITY", "TIME", "DATE", "ORDINAL"]


predicted_improved = set(predict(df_dev))
evaluation_report(spans_dev_gold, predicted_improved)

recall: 69.90028730775731%
precision: 86.005406529424%
f1 score: 77%


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [15]:
list_to_df = []
for s_id, start, end in predicted_improved:
    list_to_df.append([s_id, df_dev[df_dev['sentence_id'] == s_id].iloc[0]['sentence'], start, end, '--NME--'])
    
ner_df = pd.DataFrame(list_to_df, columns = df_dev.columns)
ner_df.head()
    

,sentence_id,sentence,beg,end,label
0,1044-001,"NABLUS , West Bank 1996-08-30",2,4,--NME--
1,1117-002,A convicted British paedophile was arrested on...,2,3,--NME--
2,1103-006,The police were given an unexpected hand by se...,25,26,--NME--
3,1038-003,""" We have the largest and most efficient port ...",46,47,--NME--
4,1142-013,Northern Iraq has been under Iraqi Kurdish con...,6,7,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [16]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for _,sentence_id, _, beg, end, label in df.itertuples():
        yield sentence_id, beg, end, label

In [17]:
mentions_dev_gold = set(gold_mentions(df_dev))

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [120]:
# TODO: Write code here to implement the baseline
def baseline_predict_mentions(df):
    for _,sentence_id, sentence, beg, end, *_ in df.itertuples():
        label = sentence.split(" ")[int(beg):int(end)]
        label = "_".join(label)
        yield sentence_id, beg, end, label
                    
predicted_mentions = set(baseline_predict_mentions(ner_df))
evaluation_report(mentions_dev_gold, predicted_mentions)

recall: 25.68869359472706%
precision: 31.60740278644209%
f1 score: 28%


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [50]:
with bz2.open("kb.tsv.bz2", 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [122]:
df_kb.loc[df_kb["mention"] == "Germany"]

,mention,entity,prob
7362,Germany,Germany,0.989291
7363,Germany,Germany_national_football_team,0.005577
7364,Germany,Nazi_Germany,0.004667
7365,Germany,Germany_Davis_Cup_team,0.000232
7366,Germany,Germany_Fed_Cup_team,0.000077
7367,Germany,German_language,0.000039
7368,Germany,East_Germany,0.000039
7369,Germany,Germany_men's_national_ice_hockey_team,0.000019
7370,Germany,Cologne,0.000019
7371,Germany,Berlin,0.000019


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [98]:
# TODO: Write code here to implement the "most probable entity" method.
def get_entity(mention):
    entity = df_kb.loc[df_kb["mention"] == mention]
    return entity.iloc[0].entity if len(entity) > 0 else "--NME--"


def knowledge_predict_mentions(df):
    for _,sentence_id, sentence, beg, end, *_ in df.itertuples():
        label = sentence.split(" ")[int(beg):int(end)]
        label = "_".join(label)
        yield sentence_id, beg, end, get_entity(label)
                    
predicted_knowledge_mentions = set(knowledge_predict_mentions(ner_df))
evaluation_report(mentions_dev_gold, predicted_knowledge_mentions)

recall: 40.40899104275815%
precision: 49.719276356830946%
f1 score: 44%


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [53]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [54]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in 1970 and the second in ...
1,1970,FIFA_World_Cup,America 1975 and during the 1970 and 1978 Worl...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the 1990 World Cup
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the 1990 W...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the 1990 World Cup The...


From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [55]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
                               ...                        
41609                                Lincoln Leyton Orient
41610               English division three Swansea Lincoln
41611    league membership narrowly edging out Lincoln ...
41612                                    Lincoln Cambridge
41613                                              Lincoln
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [138]:
# TODO: Write code here to implement the context-sensitive disambiguation method
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline


entity_dict = {}
for m in df_contexts['mention'].unique():
    prob_dist = df_kb[df_kb["mention"] == m].sort_values(by=["entity"])
    pipe = Pipeline([('cntv', CountVectorizer()), ('nbc', MultinomialNB(class_prior = list(prob_dist.prob)))])
    pipe.fit(df_contexts.context[df_contexts.mention == m], df_contexts.entity[df_contexts.mention == m])
    entity_dict[m] = pipe



In [96]:
#df_contexts.context[df_contexts.mention == "Sweden"]
df_contexts.entity[df_contexts.mention == "Sweden"].unique()[1]

'Sweden_national_football_team'

In [140]:
def get_nb_entity(mention, sentence, beg, end):
    if mention in entity_dict:
        context = sentence.split(" ")
        context = context[max(0, int(beg) - 5):min(len(context), int(end) + 5)]
        context = ' '.join(context)
        return entity_dict[mention].predict([context])[0]
    return get_entity(mention)
    

def nb_predict_mentions(df):
    for _,sentence_id, sentence, beg, end, *_ in df.itertuples():
        label = sentence.split(" ")[int(beg):int(end)]
        label = "_".join(label)
        yield sentence_id, beg, end, get_nb_entity(label, sentence, beg, end)
            
predicted_nb_mentions = set(nb_predict_mentions(ner_df))
evaluation_report(mentions_dev_gold, predicted_nb_mentions)

recall: 41.33851613993578%
precision: 50.862965273445624%
f1 score: 45%


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>